**Cargar liberias**

In [1]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import vgg16
import numpy as np

import zipfile
import os

Preparar carpeta de imágenes

In [3]:

!wget --no-check-certificate \
    "https://github.com/arbeyvillegas/IntrDeepLearning/raw/main/Entrega-02/data/brain_tumor_data.zip" \
    -O "/tmp/brain_tumor_data.zip"

zip_ref = zipfile.ZipFile('/tmp/brain_tumor_data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

--2023-11-19 17:15:54--  https://github.com/arbeyvillegas/IntrDeepLearning/raw/main/Entrega-02/data/brain_tumor_data.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/arbeyvillegas/IntrDeepLearning/main/Entrega-02/data/brain_tumor_data.zip [following]
--2023-11-19 17:15:54--  https://raw.githubusercontent.com/arbeyvillegas/IntrDeepLearning/main/Entrega-02/data/brain_tumor_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 91021706 (87M) [application/zip]
Saving to: ‘/tmp/brain_tumor_data.zip’

/tmp/brain_tumor_da 100%[===================>]  86.80M   201MB/s    in 0.4s    

2023-11-19 17:

In [4]:
carpeta_training = "/tmp/brain_tumor_data/Training"
carpeta_testing = "/tmp/brain_tumor_data/Testing"

**Cargar datos aumentados**

In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
batch_size = 32
nb_classes = 4
nb_epochs = 10
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3
img_size = 224

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(carpeta_training, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(carpeta_testing, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

Found 2870 images belonging to 4 classes.
Found 2870 images belonging to 4 classes.


In [2]:

tensorboard = TensorBoard(log_dir='./VGG16logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [10]:
steps_per_epoch = 2870//batch_size

validation_steps = 394//batch_size

In [3]:

vgg16_model = vgg16.VGG16()
vgg16_model.summary()

type(vgg16_model)

553467096/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

keras.src.engine.functional.Functional

In [6]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
     model.add(layer)
for layer in model.layers:
     layer.trainable = False
model.add(Dense(4,activation = 'softmax'))
model.compile(loss='categorical_crossentropy',optimizer= "adam", metrics=['accuracy'],)

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [11]:
#history = model.fit(X_train, Y_train,batch_size=32, epochs=20, validation_data=(X_test,Y_test), callbacks=[tensorboard])
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=nb_epochs, validation_data=validation_generator, validation_steps=validation_steps, callbacks=[tensorboard])

<ipython-input-11-e2a803a09a6a>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=nb_epochs, validation_data=validation_generator, validation_steps=validation_steps, callbacks=[tensorboard])


Epoch 1/10
89/89 [==============================] - 1987s 22s/step - loss: 1.4078 - accuracy: 0.1790 - val_loss: 1.3870 - val_accuracy: 0.2708
Epoch 2/10
89/89 [==============================] - 1944s 22s/step - loss: 1.3734 - accuracy: 0.2872 - val_loss: 1.3576 - val_accuracy: 0.3099
Epoch 3/10
89/89 [==============================] - 1944s 22s/step - loss: 1.3589 - accuracy: 0.2868 - val_loss: 1.3455 - val_accuracy: 0.2943
Epoch 4/10
89/89 [==============================] - 1946s 22s/step - loss: 1.3527 - accuracy: 0.2868 - val_loss: 1.3453 - val_accuracy: 0.3411
Epoch 5/10
89/89 [==============================] - 1944s 22s/step - loss: 1.3503 - accuracy: 0.2780 - val_loss: 1.3465 - val_accuracy: 0.2943
Epoch 6/10
89/89 [==============================] - 1938s 22s/step - loss: 1.3492 - accuracy: 0.2787 - val_loss: 1.3519 - val_accuracy: 0.3151
Epoch 7/10
89/89 [==============================] - 1939s 22s/step - loss: 1.3480 - accuracy: 0.2844 - val_loss: 1.3485 - val_accuracy: 0.2943

In [13]:
scores = model.evaluate(validation_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

90/90 [==============================] - 1750s 19s/step - loss: 1.3481 - accuracy: 0.2864
Test loss: 1.348075032234192
Test accuracy: 0.2864111363887787


In [14]:
model.save("VGG16model94accuracy_0.29-accuracy.keras")